# `MEDUSA`
aka. Dynamic-Prospective LCA aka. Union(premise, temporalis)

In [61]:
from bw_temporalis import easy_timedelta_distribution, easy_datetime_distribution, TemporalisLCA, Timeline, TemporalDistribution
from bw_temporalis.lcia import characterize_methane, characterize_co2
import bw2data as bd
import bw2calc as bc
import bw_graph_tools as graph
import numpy as np
import pandas as pd

In [62]:
bd.projects.set_current("medusa_test")

### Create some example databases
This is based on the forest example, but creating two versions


In [63]:
bd.Database('temporalis-bio').write({
    ('temporalis-bio', "CO2"): {
        "type": "emission",
        "name": "carbon dioxide",
        "temporalis code": "co2",
    },
    ('temporalis-bio', "CH4"): {
        "type": "emission",
        "name": "methane",
        "temporalis code": "ch4",
    },
})

bd.Database('dummy-database').write({
    ('dummy-database', 'dummy_activity'): {
        'name': 'Drive an electric car',
        'exchanges': [
            {
                'amount': 3,
                'type': 'technosphere',
                'input': ('dummy-database', 'dummy_activity_2'),
            },
        ]
    },
    ('dummy-database', 'dummy_activity_2'): {
        'name': 'Drive an electric car 3',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('dummy-database', 'dummy_activity_2'),
            },
            },
            {
                'amount': 3,
                'type': 'biosphere',
                'input': ('temporalis-bio', 'CO2'),
            },
        ]
    }
})


bd.Database('dummy-database-2').write({
    ('dummy-database-2', 'dummy_activity'): {
        'name': 'Drive an electric car',
        'exchanges': [
            {
                'amount': 1,
                'type': 'technosphere',
                'input': ('dummy-database-2', 'dummy_activity_2'),
            },
        ]
    },
    ('dummy-database-2', 'dummy_activity_2'): {
        'name': 'Drive an electric car 3',
        'exchanges': [
            {
                'amount': 1,
                'type': 'production',
                'input': ('dummy-database-2', 'dummy_activity_2'),
            },
            {
                'amount': 3,
                'type': 'biosphere',
                'input': ('temporalis-bio', 'CO2'),
            },
        ]
    }
})


bd.Database('temporalis-example').write({
    ('temporalis-example', 'Functional Unit'): {
        'name': 'Functional Unit',
        'exchanges': [
            {
                'amount': 5,
                'input': ('temporalis-example', 'EOL'),
                'temporal_distribution': 
                    # bwt.FixedTD(
                    # np.array(["2020-10-11", "2021-10-11", "2022-10-11", "2023-10-11", "2024-10-11"], dtype='datetime64[D]'),
                    # np.ones(5) * 1/5,
                
                    easy_timedelta_distribution(
                    start=0,
                    end=4, # Range includes both start and end
                    resolution="Y",  # M for months, Y for years, etc.
                    steps=5,
                ),
                'type': 'technosphere'
            },
        ],
    },
    ('temporalis-example', 'EOL'): {
        'exchanges': [
            {
                'amount': 0.8,
                'input': ('temporalis-example', 'Waste'),
                'type': 'technosphere'
            },
            {
                'amount': 0.2,
                'input': ('temporalis-example', 'Landfill'),
                'type': 'technosphere'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Use'),
                'type': 'technosphere'
            },
        ],
        'name': 'EOL',
        'type': 'process'
    },
    ('temporalis-example', 'Use'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Production'),
                'temporal_distribution': TemporalDistribution(
                    np.array([-4], dtype='timedelta64[M]'),
                    np.array([1.0])
                ),
                'type': 'technosphere'
            },
        ],
        'name': 'Use',
    },
    ('temporalis-example', 'Production'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Transport'),
                'temporal_distribution': TemporalDistribution(
                    np.array([200],dtype='timedelta64[D]'),
                    np.array([1.0])
                ),
                'type': 'technosphere'
            },
        ],
        'name': 'Production',
        'type': 'process'
    },
    ('temporalis-example', 'Transport'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Sawmill'),
                'type': 'technosphere'
            },
            {
                'amount': 0.1,
                'input': ('temporalis-bio', 'CO2'),
                'type': 'biosphere'
            },
        ],
        'name': 'Production',
        'type': 'process'
    },
    ('temporalis-example', 'Sawmill'): {
        'exchanges': [
            {
                'amount': 1.2,
                'input': ('temporalis-example', 'Forest'),
                'temporal_distribution': TemporalDistribution(
                    np.array([-14], dtype='timedelta64[M]'),
                    np.array([1.0])
                ),
                'type': 'technosphere'
            },
            {
                'amount': 0.1,
                'input': ('temporalis-bio', 'CO2'),
                'type': 'biosphere'
            },
            {
                'amount': 4,
                'input': ('dummy-database', 'dummy_activity'),
                'temporal_distribution': TemporalDistribution(
                    np.array([-5], dtype='timedelta64[M]'),
                    np.array([1.0])
                ),
                'type': 'technosphere'
            },
        ],
        'name': 'Sawmill',
        'type': 'process'
    },
    ('temporalis-example', 'Forest'): {
        'exchanges': [
            {
                'amount': -.1 * 6,
                'input': ('temporalis-bio', 'CO2'),
                'temporal_distribution': TemporalDistribution(
                    np.array([-4, -3, 0, 1, 2, 5], dtype='timedelta64[Y]'),
                    np.ones(6) * (1/6)
                ),
                'type': 'biosphere'
            },
            {
                'amount': 1.5,
                'input': ('temporalis-example', 'Thinning'),
                'temporal_distribution': TemporalDistribution(
                    np.array([-3, 0, 1], dtype='timedelta64[Y]'),
                    np.ones(3) * 1/3
                ),
                'type': 'technosphere'
            },
        ],
        'name': 'Forest',
    },
    ('temporalis-example', 'Thinning'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Thinning'),
                'type': 'production'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Avoided impact - thinnings'),
                'type': 'production'
            },
        ],
        'name': 'Thinning',
        'type': 'process'
    },
    ('temporalis-example', 'Landfill'): {
        'exchanges': [
            {
                'amount': 0.1,
                'input': ('temporalis-bio', 'CH4'),
                'temporal_distribution': TemporalDistribution(
                    np.array([10, 20, 60, 100], dtype='timedelta64[M]'),
                    np.ones(4) * 1/4
                ),
                'type': 'biosphere'
            },
        ],
        'name': 'Landfill',
        'type': 'process'
    },
    ('temporalis-example', 'Waste'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Waste'),
                'type': 'production'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Avoided impact - waste'),
                'type': 'production'
            },
        ],
        'name': 'Waste',
        'type': 'process'
    },
    ('temporalis-example', 'Avoided impact - waste'): {
        'exchanges': [
            {
                'amount': -0.4,
                'input': ('temporalis-bio', 'CO2'),
                'type': 'biosphere'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Avoided impact - waste'),
                'type': 'production'
            },
        ],
        'name': 'Avoided impact - waste',
        'type': 'process'
    },
    ('temporalis-example', 'Avoided impact - thinnings'): {
        'exchanges': [
            {
                'amount': -0.3,
                'input': ('temporalis-bio', 'CO2'),
                'type': 'biosphere'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Avoided impact - thinnings'),
                'type': 'production'
            },
        ],
        'name': 'Avoided impact - thinnings',
        'type': 'process'
    }
})


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<00:00, 2020.86it/s]

Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 12/12 [00:00<?, ?it/s]

Vacuuming database 


In [64]:
bd.Method(("GWP", "example")).write([
    (('temporalis-bio', "CO2"), 1),
    (('temporalis-bio', "CH4"), 25),
])

In [65]:
lca = bc.LCA({('temporalis-example', 'EOL'): 1}, ("GWP", "example"))
lca.lci()
lca.lcia()
lca.score

RuntimeError: failed to factorize matrix at line 413 in file ../scipy/sparse/linalg/_dsolve/SuperLU/SRC/dpanel_bmod.c


# Custom `EdgeExtractor` class

In [ ]:
from dataclasses import dataclass
from heapq import heappop, heappush
from typing import Callable, List

from bw_temporalis import TemporalisLCA, TemporalDistribution


@dataclass
class Edge:
    """
    Class for storing a temporal edge with source and target.

    Leaf edges link to a source process which is a leaf in
    our graph traversal (either through cutoff or a filter
    function).

    Attributes
    ----------
    distribution : TemporalDistribution
    leaf : bool
    consumer : int
    producer : int

    """

    distribution: TemporalDistribution
    leaf: bool
    consumer: int
    producer: int


class EdgeExtracter(TemporalisLCA):
    def __init__(self, *args, edge_filter_function: Callable = None, **kwargs):
        super().__init__(*args, **kwargs)
        if edge_filter_function:
            self.edge_ff = edge_filter_function
        else:
            self.edge_ff = lambda x: False

    def build_edge_timeline(self, node_timeline: bool | None = False) -> List:
        heap = []
        timeline = []

        for edge in self.edge_mapping[self.unique_id]:
            node = self.nodes[edge.producer_unique_id]
            heappush(
                heap,
                (
                    1 / node.cumulative_score,
                    self.t0 * edge.amount,
                    node,
                ),
            )
            timeline.append(
                Edge(
                    distribution=self.t0 * edge.amount,
                    leaf=False,
                    consumer=self.unique_id,
                    producer=node.activity_datapackage_id,
                )
            )

        while heap:
            _, td, node = heappop(heap)

            for edge in self.edge_mapping[node.unique_id]:
                row_id = self.nodes[edge.producer_unique_id].activity_datapackage_id
                col_id = node.activity_datapackage_id
                exchange = self.get_technosphere_exchange(
                    input_id=row_id,
                    output_id=col_id,
                )
                value = (
                    self._exchange_value(
                        exchange=exchange,
                        row_id=row_id,
                        col_id=col_id,
                        matrix_label="technosphere_matrix",
                    )
                    / node.reference_product_production_amount
                )
                producer = self.nodes[edge.producer_unique_id]
                leaf = self.edge_ff(row_id)

                distribution = (td * value).simplify()
                timeline.append(
                    Edge(
                        distribution=distribution,
                        leaf=leaf,
                        consumer=node.activity_datapackage_id,
                        producer=producer.activity_datapackage_id,
                    )
                )
                if not leaf:
                    heappush(
                        heap,
                        (
                            1 / node.cumulative_score,
                            distribution,
                            producer,
                        ),
                    )
        return timeline

In [ ]:
SKIPPABLE = [
    node.id for node in bd.Database('dummy-database')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

In [ ]:
eelca = EdgeExtracter(lca, edge_filter_function=filter_function)

Starting graph traversal
Calculation count: 12


In [ ]:
timeline = eelca.build_edge_timeline()

In [ ]:
from datetime import datetime
dates_list = [
        datetime.strptime("2019", "%Y"),
        datetime.strptime("2023", "%Y"),
    ]

In [ ]:
from utils import create_grouped_edge_dataframe, get_datapackage_from_edge_timeline
timeline_df = create_grouped_edge_dataframe(timeline, dates_list, "linear")

In [ ]:
database_date_dict = {
            2019: 'dummy-database',
            2023: 'dummy-database-2',
        }

In [ ]:
dp = get_datapackage_from_edge_timeline(timeline_df, database_date_dict)

2023 dummy-database-2 Drive an electric car
129 138 131 138002023


In [ ]:
demand = {('temporalis-example', 'EOL'):1}
gwp = ('GWP', 'example')

In [ ]:
fu, data_objs, remapping = bd.prepare_lca_inputs(demand=demand, method=gwp)

In [18]:
lca = bc.LCA(fu, data_objs = data_objs + [dp], remapping_dicts=remapping)
lca.lci()
lca.lcia()
lca.score

In [48]:
lca.load_lci_data(nonsquare_ok=True)

In [57]:
set(lca.dicts.activity.reversed.items())

{(0, 129),
 (1, 130),
 (2, 133),
 (3, 134),
 (4, 135),
 (5, 136),
 (6, 137),
 (7, 138),
 (8, 139),
 (9, 140),
 (10, 141),
 (11, 142),
 (12, 143),
 (13, 144),
 (14, 1002023),
 (15, 129002023),
 (16, 134002023),
 (17, 135002023),
 (18, 136002023),
 (19, 137002023),
 (20, 138002022),
 (21, 138002023),
 (22, 139002019),
 (23, 139002022),
 (24, 139002023),
 (25, 140002019),
 (26, 140002022),
 (27, 140002023),
 (28, 141002023),
 (29, 142002023),
 (30, 143002023),
 (31, 144002019),
 (32, 144002022),
 (33, 144002023)}

In [59]:
bd.get_node(id=131)['database']

'dummy-database-2'

In [53]:
prods = set(lca.dicts.product.reversed.items())
prods

{(0, 129),
 (1, 130),
 (2, 131),
 (3, 133),
 (4, 134),
 (5, 135),
 (6, 136),
 (7, 137),
 (8, 138),
 (9, 139),
 (10, 140),
 (11, 141),
 (12, 142),
 (13, 143),
 (14, 144),
 (15, 1002023),
 (16, 129002023),
 (17, 134002023),
 (18, 135002023),
 (19, 136002023),
 (20, 137002023),
 (21, 138002022),
 (22, 138002023),
 (23, 139002019),
 (24, 139002022),
 (25, 139002023),
 (26, 140002019),
 (27, 140002022),
 (28, 140002023),
 (29, 141002023),
 (30, 142002023),
 (31, 143002023),
 (32, 144002019),
 (33, 144002022),
 (34, 144002023)}

In [54]:
set.difference(acts, prods)

{(2, 133),
 (3, 134),
 (4, 135),
 (5, 136),
 (6, 137),
 (7, 138),
 (8, 139),
 (9, 140),
 (10, 141),
 (11, 142),
 (12, 143),
 (13, 144),
 (14, 1002023),
 (15, 129002023),
 (16, 134002023),
 (17, 135002023),
 (18, 136002023),
 (19, 137002023),
 (20, 138002022),
 (21, 138002023),
 (22, 139002019),
 (23, 139002022),
 (24, 139002023),
 (25, 140002019),
 (26, 140002022),
 (27, 140002023),
 (28, 141002023),
 (29, 142002023),
 (30, 143002023),
 (31, 144002019),
 (32, 144002022),
 (33, 144002023)}

In [ ]:
lca.technosphere_matrix.toarray()

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-3.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -4.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -5.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        , 

# Static Case

In [24]:
lca = bc.LCA({('temporalis-example', 'EOL'): 1}, ("GWP", "example"))
lca.lci()
lca.lcia()
lca.score

0.0

In [20]:
all_acts = [act for db in bd.databases for act in bd.Database(db)]
print(all_acts)
print(len(all_acts))

['carbon dioxide' (None, None, None), 'methane' (None, None, None), 'Drive an electric car' (None, None, None), 'Drive an electric car 3' (None, None, None), 'Drive an electric car 3' (None, None, None), 'Drive an electric car' (None, None, None), 'Avoided impact - waste' (None, None, None), 'EOL' (None, None, None), 'Thinning' (None, None, None), 'Sawmill' (None, None, None), 'Production' (None, None, None), 'Waste' (None, None, None), 'Forest' (None, None, None), 'Avoided impact - thinnings' (None, None, None), 'Landfill' (None, None, None), 'Functional Unit' (None, None, None), 'Use' (None, None, None), 'Production' (None, None, None)]
18
